# Dataset simpel
In this notebook we will try to create batched with slices of data.
This should bethe final dataform which we will need. 
We will create a tensort with x timepoints with y slices and z observations (x,y,z)



In [1]:
%load_ext autoreload
%autoreload 2
from sys import path
import tensorflow as tf
from scipy.io import arff

path.append('..')

2022-06-13 19:19:41.879837: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-13 19:19:41.879883: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from src.data.make_dataset import get_eeg
datapath = get_eeg(data_dir="../data/raw")
path = "../data/raw"
from src.data.make_dataset import get_eeg_data
data = get_eeg_data(datapath)

1712128/1696428 [==============================] - 1s 1us/step


2022-06-13 19:20:01.418 | INFO     | src.data.make_dataset:get_eeg:27 - Data is downloaded to /tmp/.keras/datasets/eeg.
2022-06-13 19:20:01.419 | INFO     | src.data.make_dataset:get_eeg_data:34 - /tmp/.keras/datasets/eeg already exists, skipping download


The chunk iterator we use now uses an iterator to get data from the dataset. It will stop when the obervation changes.
We will also padding to create symetric datasets.


In [3]:
from src.data.make_grouped_chunked_eeg_data import EEGChunkIterator
from src.data.make_eeg_data import EEGListDataset

dsNew = EEGListDataset(data=data[0].tolist())
dataloader = EEGChunkIterator(dsNew, batchsize=2, horizon=3)


We know the data contains 14 observations, with a batchsize of 2 and a horizon of 3 we expect a tensort with the shape [2,3,14]

In [4]:
# gets mostly even sized chunks of 'batch size' 

x = next(iter(dataloader))
x.shape

torch.Size([2, 3, 14])

Great success! We get the correct tensor. Not let's check if the slicing is correct

In [5]:
# with the sliding windows the second window of the first batch 
# should be the same as the first window of the second batch
x[0][1]

tensor([4327.6900, 4006.6700, 4295.3800, 4156.4100, 4336.9200, 4583.5900,
        4096.9200, 4630.2600, 4207.6900, 4222.0500, 4206.6700, 4282.0500,
        4628.7200, 4389.2300], dtype=torch.float64)

In [6]:
x[1][0]
# and this checks out!

tensor([4327.6900, 4006.6700, 4295.3800, 4156.4100, 4336.9200, 4583.5900,
        4096.9200, 4630.2600, 4207.6900, 4222.0500, 4206.6700, 4282.0500,
        4628.7200, 4389.2300], dtype=torch.float64)

In [7]:
for i in range(0,12):
    x = next(iter(dataloader))
    print(x.shape)

torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])


All sensors seem to be the right size, so nice.
However, we want to improve on the performance by buffering instead of iteration in the next notebook